In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import preprocessing

In [2]:
train_df = pd.read_csv("/home/jo/Python projects/Projets/Compétition Kaggle/train.csv")
test_df = pd.read_csv("/home/jo/Python projects/Projets/Compétition Kaggle/test.csv")

In [3]:
train_df["Total_expense"] = train_df.iloc[:, -7:-2].sum(axis=1)
train_df["Spent_money"] = train_df["Total_expense"].apply(lambda x: False if x == 0 else True)

test_df["Total_expense"] = test_df.iloc[:, -7:-2].sum(axis=1)
test_df["Spent_money"] = test_df["Total_expense"].apply(lambda x: False if x == 0 else True)

In [4]:
train_df.Cabin.fillna("Z/6666/Z", inplace=True)
test_df.Cabin.fillna("Z/6666/Z", inplace=True)

train_df["Cabin_num"] = train_df["Cabin"].apply(lambda x: x.split("/")[1]).astype(int)
test_df["Cabin_num"] = test_df["Cabin"].apply(lambda x: x.split("/")[1]).astype(int)

train_df["Deck"] = train_df["Cabin"].apply(lambda x: x[0])
train_df["Side"] = train_df["Cabin"].apply(lambda x: x[-1])

test_df["Deck"] = test_df["Cabin"].apply(lambda x: x[0])
test_df["Side"] = test_df["Cabin"].apply(lambda x: x[-1])


train_df["Deck"].loc[train_df.Deck == "T"] = "A"
test_df["Deck"].loc[test_df.Deck == "T"] = "A"


train_df.Deck.replace("Z", np.nan, inplace=True)
test_df.Deck.replace("Z", np.nan, inplace=True)

train_df.Side.replace("Z", np.nan, inplace=True)
test_df.Side.replace("Z", np.nan, inplace=True)

/home/jo/miniconda3/envs/johnny_lerouge/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
y_train = train_df.Transported

In [6]:
bool_var = ["CryoSleep", "Spent_money"]
continuous_var = [
    "Age",
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck",
    "Total_expense"
]
categorical_var = ["HomePlanet", "Destination", "Deck", "Side"]

In [7]:
features = continuous_var + categorical_var + bool_var

In [8]:
for var in bool_var:
    train_df[var] = train_df[var].map({False: 0, True: 1})
    test_df[var] = test_df[var].map({False: 0, True: 1})

In [9]:
for col in continuous_var:
    train_df[col] = np.log1p(train_df[col])
    test_df[col] = np.log1p(test_df[col])

In [10]:
from category_encoders import CatBoostEncoder

CBE = CatBoostEncoder(handle_missing="return_nan")

In [11]:
for col in categorical_var:
    train_df[col] = CBE.fit_transform(train_df[col], y_train)
    test_df[col] = CBE.transform(test_df[col])

In [12]:
encoded_df_train = train_df[bool_var + continuous_var + categorical_var]
encoded_df_test = test_df[bool_var + continuous_var + categorical_var]

In [13]:
train_df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Total_expense,Spent_money,Cabin_num,Deck,Side
0,0001_01,0.503624,0.0,B/0/P,0.503624,3.688879,False,0.000000,0.000000,0.000000,0.000000,0.000000,Maham Ofracculy,False,0.000000,0,0,0.503624,0.503624
1,0002_01,0.503624,0.0,F/0/S,0.251812,3.218876,False,4.700480,2.302585,3.258097,6.309918,3.806662,Juanna Vines,True,6.602588,1,0,0.503624,0.503624
2,0003_01,0.251812,0.0,A/0/S,0.501208,4.077537,True,3.784190,8.182280,0.000000,8.812248,3.912023,Altark Susent,False,9.248021,1,0,0.503624,0.751812
3,0003_02,0.167875,0.0,A/0/S,0.375906,3.526361,False,0.000000,7.157735,5.918894,8.110728,5.267858,Solam Susent,False,8.551981,1,0,0.251812,0.501208
4,0004_01,0.751812,0.0,F/1/S,0.300725,2.833213,False,5.717028,4.262680,5.023881,6.338594,1.098612,Willy Santantines,True,6.995766,1,1,0.751812,0.375906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.659231,0.0,A/98/P,0.610619,3.737670,True,0.000000,8.827615,0.000000,7.404888,4.317488,Gravior Noxnuther,False,9.052165,1,98,0.492351,0.451380
8689,9278_01,0.423930,1.0,G/1499/S,0.504402,2.944439,False,0.000000,0.000000,0.000000,0.000000,0.000000,Kurta Mondalley,False,0.000000,0,1499,0.516225,0.555077
8690,9279_01,0.423838,0.0,G/1500/S,0.471002,3.295837,False,0.000000,0.000000,7.535297,0.693147,0.000000,Fayey Connon,True,7.535830,1,1500,0.516023,0.554947
8691,9280_01,0.658922,0.0,E/608/S,0.610280,3.496508,False,0.000000,6.956545,0.000000,5.869297,8.082093,Celeon Hontichre,False,8.442039,1,608,0.357147,0.555051


In [14]:
encoded_df_train

,CryoSleep,Spent_money,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Total_expense,HomePlanet,Destination,Deck,Side
0,0.0,0,3.688879,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.503624,0.503624,0.503624,0.503624
1,0.0,1,3.218876,4.700480,2.302585,3.258097,6.309918,3.806662,6.602588,0.503624,0.251812,0.503624,0.503624
2,0.0,1,4.077537,3.784190,8.182280,0.000000,8.812248,3.912023,9.248021,0.251812,0.501208,0.503624,0.751812
3,0.0,1,3.526361,0.000000,7.157735,5.918894,8.110728,5.267858,8.551981,0.167875,0.375906,0.251812,0.501208
4,0.0,1,2.833213,5.717028,4.262680,5.023881,6.338594,1.098612,6.995766,0.751812,0.300725,0.751812,0.375906
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,1,3.737670,0.000000,8.827615,0.000000,7.404888,4.317488,9.052165,0.659231,0.610619,0.492351,0.451380
8689,1.0,0,2.944439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.423930,0.504402,0.516225,0.555077
8690,0.0,1,3.295837,0.000000,0.000000,7.535297,0.693147,0.000000,7.535830,0.423838,0.471002,0.516023,0.554947
8691,0.0,1,3.496508,0.000000,6.956545,0.000000,5.869297,8.082093,8.442039,0.658922,0.610280,0.357147,0.555051


In [15]:
# On impute les valeurs manquantes grâce à un modèle qui se base sur les forêts aléatoires

import miceforest as mf
import random
import sklearn.neighbors._base
import sys

sys.modules["sklearn.neighbors.base"] = sklearn.neighbors._base
from missingpy import MissForest

imputer = MissForest(n_estimators=200)
encoded_df_train = imputer.fit_transform(encoded_df_train)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


In [16]:
imputer = MissForest(n_estimators=200)
encoded_df_test = imputer.fit_transform(encoded_df_test)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [17]:
encoded_df_train = pd.DataFrame(encoded_df_train, columns=features)

In [18]:
encoded_df_test = pd.DataFrame(encoded_df_test, columns=features)

In [19]:
encoded_df_train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Total_expense,HomePlanet,Destination,Deck,Side,CryoSleep,Spent_money
0,0.0,0.0,3.688879,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.503624,0.503624,0.503624,0.503624
1,0.0,1.0,3.218876,4.700480,2.302585,3.258097,6.309918,3.806662,6.602588,0.503624,0.251812,0.503624,0.503624
2,0.0,1.0,4.077537,3.784190,8.182280,0.000000,8.812248,3.912023,9.248021,0.251812,0.501208,0.503624,0.751812
3,0.0,1.0,3.526361,0.000000,7.157735,5.918894,8.110728,5.267858,8.551981,0.167875,0.375906,0.251812,0.501208
4,0.0,1.0,2.833213,5.717028,4.262680,5.023881,6.338594,1.098612,6.995766,0.751812,0.300725,0.751812,0.375906
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,1.0,3.737670,0.000000,8.827615,0.000000,7.404888,4.317488,9.052165,0.659231,0.610619,0.492351,0.451380
8689,1.0,0.0,2.944439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.423930,0.504402,0.516225,0.555077
8690,0.0,1.0,3.295837,0.000000,0.000000,7.535297,0.693147,0.000000,7.535830,0.423838,0.471002,0.516023,0.554947
8691,0.0,1.0,3.496508,0.000000,6.956545,0.000000,5.869297,8.082093,8.442039,0.658922,0.610280,0.357147,0.555051


In [20]:
import tpot
from tpot import TPOTClassifier
from sklearn.model_selection import StratifiedKFold

tpot = TPOTClassifier(
    generations=5,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0),
    scoring="accuracy",
    population_size=750,
    random_state=69,
    verbosity=2,
)

tpot.fit(encoded_df_train, y_train)

Optimization Progress:   0%|          | 0/4500 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8067405112920868

Generation 2 - Current best internal CV score: 0.8067405112920868

Generation 3 - Current best internal CV score: 0.8072002811027857


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), bootstrap=True, criterion=gini, max_features=0.4, min_samples_leaf=10, min_samples_split=8, n_estimators=100)


TPOTClassifier(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
               generations=5, population_size=750, random_state=69,
               scoring='accuracy', verbosity=2)

In [31]:
tpot

TPOTClassifier(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
               generations=5, population_size=750, random_state=69,
               scoring='accuracy', verbosity=2)

In [32]:
from sklearn.metrics import accuracy_score

accuracy_score(tpot.predict(encoded_df_train), y_train)

0.8433222132750489

In [23]:
from sklearn.metrics import f1_score

f1_score(tpot.predict(encoded_df_train), y_train)

0.8428703276419012

In [24]:
from sklearn.metrics import roc_auc_score

roc_auc_score(tpot.predict(encoded_df_train), y_train)

0.8434273463685228

In [33]:
predictions = tpot.predict(encoded_df_test)

In [34]:
passenger = test_df.PassengerId

In [35]:
def make_submission(preds):
    sub_data=np.column_stack([passenger, preds])
    submission=pd.DataFrame(columns=['PassengerId', 'Transported' ], data=sub_data)
    return submission

In [36]:
final=make_submission(predictions)

In [37]:
final

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [38]:
final.to_csv('/home/jo/Téléchargements/final.csv',index=False)

## Documentation

Notebook m'ayant servi comme base de travail : https://storage.googleapis.com/kaggle-forum-message-attachments/1838122/17938/spaceship-titanic%20(2).ipynb

Lien vers le kernel kaggle que j'ai publié : https://www.kaggle.com/code/johnnylerouge/spaceship-titanic-avec-tpot